# Evaluación de los modelos

En este documento se muestran las pruebas realizadas para la evaluación del modelo mediante el conjunto de entrenamiento. Se pondrán a pruebas los modelos original y cuantizado de los 3 problemas de aprendizaje empleando la partición de test reservada antes del preprocesado de datos.

## Resultados de test para clasificación binaria

Se procede al cargado en memoria del modelo previamente entrenado para el entrenamiento habitual de escritorio. Es necesario importar el directorio de ficheros de entrenamiento para reconstruir el modelo, y posteriormente, realizar la inferencia de test. El flujo de código resultado es similar al entrenamiento, con la diferencia de que en lugar de optimizar el modelo, se procede directamente a la inferencia de los valores de test empleando los pesos ya entrenados.

Otra opción de cargar el modelo sería utilizar el formato torchscript, pero en clasificación binaria, debido a la magnitud de los datos, es más favorecedor utilizar el objeto pth. Para la clasificación maligna y benigna, si se empleará este formato.

Comenzamos por la importación de librerías

In [1]:
# Librerías utilizadas por el script

import os
#os.environ['HF_HOME'] = '/mnt/homeGPU/hexecode/cache/'

import timm
import numpy as np
import pandas as pd
import cv2
import torch
import fastbook
import fastai
import fastcore
import PIL
import shutil
import albumentations
import torch
from torchvision import transforms
from PIL import Image
from datetime import datetime

import matplotlib.pyplot as plt
import imgaug as ia
import seaborn as sns

from imgaug import augmenters as iaa
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from fastai.vision.all import *
from nbdev.showdoc import *
from fastai.vision.all import *
import torch.utils.mobile_optimizer as mobile_optimizer
from torch.utils.mobile_optimizer import optimize_for_mobile
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchvision.models import resnet18
from torchcam.methods import SmoothGradCAMpp
import torch

"""
!pip install -Uqq fastbook
!pip install nbdev
"""
fastbook.setup_book()
torch.cuda.is_available()
TEST_DIR = "testThumbnails/"
test_metadata = pd.read_csv('testSet.csv')


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.9 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations


### Modelo original
Evaluamos el conjunto de test con el modelo original

In [2]:
#Preprocesamiento (solo resize y normalización)
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_imgs = test_metadata['image'].to_list()
y_true = test_metadata['bin'].to_list()

In [10]:
def evaluate_original(model, test_imgs):
    y_pred = []
    for img in test_imgs:
        # Cargar imagen
        image_path = TEST_DIR + img
        image = Image.open(image_path)
        input_tensor = transform(image).unsqueeze(0).cuda()

        # Realiza la predicción
        with torch.no_grad():
            output = model(input_tensor)

        # Indice con la mayor probabilidad
        _, predicted_class = output.max(1)

        y_pred.append(predicted_class.item())

    return y_pred

In [4]:
# Cargamos el modelo original
model = torch.jit.load("bestISICFocal_512_32_50.pt").to('cuda')
y_pred = evaluate_original(model, test_imgs)

Mostramos los resultados en un formato legible visible por el usuario.

In [5]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.89      0.89     16010
           1       0.68      0.70      0.69      5468

    accuracy                           0.84     21478
   macro avg       0.79      0.80      0.79     21478
weighted avg       0.84      0.84      0.84     21478



Ahora, repetimos el mismo proceso para el modelo adaptado para android. Realziaremos la evaluación con el modelo estándar sólo transformado al formato android, y el modelo cuantizado optimizado.

### Modelo cuantizado

In [6]:
#Cargamos el modelo
model_opt = torch.jit.load("bestISICFocal512_32_50_android.ptl")

Cargamos la lista de imágenes de test e iteramos por ellas, añadiendo la predicción al resultado.

In [46]:
def evaluate_mobile(model, test_imgs):
    y_pred = []
    for img in test_imgs:
        # Cargar imagen
        image_path = TEST_DIR + img
        image = Image.open(image_path)
        input_tensor = transform(image).unsqueeze(0)


        # Realiza la predicción
        with torch.no_grad():
            output = model(input_tensor)


        # Indice con la mayor probabilidad
        _, predicted_class = output.max(1)

        y_pred.append(predicted_class.item())

    return y_pred

In [8]:
y_pred_mobile = evaluate_mobile(model_opt, test_imgs)

In [9]:
print(classification_report(y_true, y_pred_mobile))

              precision    recall  f1-score   support

           0       0.90      0.89      0.89     16010
           1       0.68      0.70      0.69      5468

    accuracy                           0.84     21478
   macro avg       0.79      0.80      0.79     21478
weighted avg       0.84      0.84      0.84     21478



Para verificar en detalle el correcto funcionamiento del modelo, podemos emplear GradCam, y observar en qué puntos de la imagen se centra el modelo. Comenzaremos por una imagen correctamente clasificada.
** NO DISPONIBLE PARA MODELO OPTIMIZADO O CUANTIZADO**

## Resultados de test para clasificación multiclase

Ahora, evaluaremos el rendimiento de los dos modelos de clasificación multiclase. Ambos modelos serán evaluados con la partición de test, extrayendo las clases oportunas, y aplicando el mismo tipo de preprocesado que al conjunto de entrenamiento.

### Modelo de clases malignas

En este primer caso, estudiaremos el impacto en métricas del modelo original vs cuantizado en test, empleando unicamente imágenes malignas. Recordar que, en este modelo, debemos realizar el mismo preprocesado que en train, por lo que debemos:
- Fusionar la clase melanoma y melanoma metastasis
- Normalizar y redimensionar usando los parámetros de imagenet y resolución 512 x 512.

#### Modelo original

In [49]:
train_malignant = test_metadata.loc[test_metadata['bin'] == 1]
train_malignant['label'].replace('melanoma_metastasis', 'melanoma', inplace=True)

SettingWithCopyError: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [50]:
#Renombramos a notación numérica
replacements = {'basal_cell_carcinoma':0,'melanoma': 1,'squamous_cell_carcinoma':2}
train_malignant['label'] = train_malignant['label'].map(replacements).fillna(train_malignant['label'])

SettingWithCopyError: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

Comprobamos que la transformación ha sido exitosa

In [51]:
print(train_malignant['label'].unique())
print(train_malignant.label.value_counts())
y_true = train_malignant.label.to_list()
test_malign_imgs = train_malignant.image.to_list()

[1 0 2]
label
1    2950
0    1969
2     549
Name: count, dtype: int64


Cargamos modelo original y realizamos inferncia

In [52]:
#Preprocesamiento (solo resize y normalización)
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [53]:
# Cargamos el modelo original
model = torch.jit.load("bestISICmalignant_512.pt").to('cuda')
y_pred = evaluate_original(model, test_malign_imgs)

In [54]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.84      0.82      1969
           1       0.87      0.92      0.89      2950
           2       0.65      0.37      0.47       549

    accuracy                           0.83      5468
   macro avg       0.78      0.71      0.73      5468
weighted avg       0.83      0.83      0.83      5468



### Modelo cuantizado

In [55]:
model_opt = torch.jit.load("bestISICmalignant 512_android.ptl")

In [56]:
y_pred = evaluate_mobile(model_opt, test_malign_imgs)

In [57]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.84      0.82      1969
           1       0.87      0.92      0.89      2950
           2       0.65      0.37      0.47       549

    accuracy                           0.83      5468
   macro avg       0.78      0.71      0.73      5468
weighted avg       0.82      0.83      0.82      5468

